In [29]:

import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer

# Load the datasets
sentences_df = pd.read_csv('/kaggle/input/nlp-project/sentences.csv')  # Replace with actual file path if needed
data_df = pd.read_csv('/kaggle/input/nlp-project/data (1).csv')  # Replace with actual file path

# Preview datasets
print(sentences_df.head())
print(data_df.head())


                                   darija  \
0    homa mkhbbyin chi haja, ana mti99en!   
1      bayna homa tay7awlo ib9aw mbrrdin.   
2  loTilat mabaynach fihom mori7in bzzaf.   
3      ghaliban ghayjrriw 3lih mn lkhdma!   
4                     Tab3an rah mkta2eb!   

                                                 eng  \
0                They're hiding something, I'm sure!   
1    It's obvious they're trying to keep their cool.   
2            the hotels don't seem very comfortable.   
3  he is probably about to be laid off by head of...   
4                         of course he's depressive!   

                               darija_ar  
0      هوما مخبّيين شي حاجة, أنا متيقّن!  
1     باينا هوما تايحاولو إبقاو مبرّدين.  
2  لوطيلات مابايناش فيهوم موريحين بزّاف.  
3        غاليبان غايجرّيو عليه من لخدما!  
4                     طابعان راه مكتاءب!  
         darija
0    lbs hwayjk
1     kayn lbrd
2    skhon lhal
3    kaytih tlj
4  ghadi nsafro


In [30]:

class VanillaLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(VanillaLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, (hn, cn) = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Take the last time step output
        return out


In [31]:

# Use the correct columns
sentences = sentences_df['darija']  # Source text in Darija
labels = sentences_df['eng']  # Target translation in English

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split into train and test sets
train_sentences, test_sentences, train_labels, test_labels = train_test_split(
    sentences, labels_encoded, test_size=0.2, random_state=42
)

print(f"Train size: {len(train_sentences)}, Test size: {len(test_sentences)}")


Train size: 70228, Test size: 17557


In [32]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

In [33]:
# 1. Chargement des données
sentences_df = pd.read_csv("/kaggle/input/nlp-project/sentences.csv")  # Remplacez par le bon chemin

# 2. Préparer les colonnes
sentences = sentences_df['darija']  # Texte source
labels = sentences_df['eng']  # Traductions cibles

# 3. Remplir les valeurs manquantes
sentences = sentences.fillna('')
labels = labels.fillna('')

# 4. Diviser en ensembles d'entraînement et de test
train_sentences, test_sentences, train_labels, test_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42
)

# 5. Convertir les labels en listes
train_labels_text = list(train_labels)
test_labels_text = list(test_labels)


In [34]:
print("Exemple d'étiquettes d'entraînement :", train_labels_text[:10])
print("Exemple d'étiquettes de test :", test_labels_text[:10])

Exemple d'étiquettes d'entraînement : ['', '', "I'm gonna make cake for dessert", 'You gave me an idea', '', '', '', '', '', '']
Exemple d'étiquettes de test : ['where is the coolest place you traveled?', '', '', '', '', '', '', '', '', '']


In [35]:
# Filtrer les étiquettes pour s'assurer qu'elles sont des chaînes valides
train_labels_text = [str(label).strip() for label in train_labels_text if isinstance(label, str) and label.strip()]
test_labels_text = [str(label).strip() for label in test_labels_text if isinstance(label, str) and label.strip()]


In [36]:
train_labels_encoded = tokenizer(list(train_labels_text), padding=True, truncation=True, return_tensors="pt")
test_labels_encoded = tokenizer(list(test_labels_text), padding=True, truncation=True, return_tensors="pt")

print("Tokenisation réussie !")


Tokenisation réussie !


In [37]:
print(train_labels_encoded['input_ids'].shape)
print(test_labels_encoded['input_ids'].shape)


torch.Size([10207, 57])
torch.Size([2536, 37])


In [38]:
import numpy as np

# Remplacer les NaN par une chaîne vide
train_labels_text = [label if isinstance(label, str) else '' for label in train_labels_text]
test_labels_text = [label if isinstance(label, str) else '' for label in test_labels_text]

# Éliminer les étiquettes vides
train_labels_text = [label for label in train_labels_text if label.strip()]
test_labels_text = [label for label in test_labels_text if label.strip()]

print("Nettoyage terminé. Échantillons après traitement :")
print("Train labels:", train_labels_text[:10])
print("Test labels:", test_labels_text[:10])


Nettoyage terminé. Échantillons après traitement :
Train labels: ["I'm gonna make cake for dessert", 'You gave me an idea', 'With eggs?', "Waitresses aren't interesting", 'where did you hear this?', 'There you go', 'what are they?', "i'm trying a new recipe for dinner", "i wouldn't recommend the meat", 'the book is behind the table']
Test labels: ['where is the coolest place you traveled?', 'Congratulations on your promotion', "I don't want it", 'What ingredients?', "it's ok", 'At last!', 'I have been to Europe twice', 'take care', 'february', "i don't like this tea"]


In [39]:
assert all(isinstance(label, str) for label in train_labels_text), "Problème dans les étiquettes d'entraînement"
assert all(isinstance(label, str) for label in test_labels_text), "Problème dans les étiquettes de test"


In [40]:
# Tokeniser les étiquettes nettoyées
train_labels_encoded = tokenizer(train_labels_text, padding=True, truncation=True, return_tensors="pt")
test_labels_encoded = tokenizer(test_labels_text, padding=True, truncation=True, return_tensors="pt")

print("Tokenisation réussie !")
print(f"Taille des étiquettes tokenisées d'entraînement : {train_labels_encoded['input_ids'].shape}")
print(f"Taille des étiquettes tokenisées de test : {test_labels_encoded['input_ids'].shape}")


Tokenisation réussie !
Taille des étiquettes tokenisées d'entraînement : torch.Size([10207, 57])
Taille des étiquettes tokenisées de test : torch.Size([2536, 37])


In [43]:

# Initialiser le tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# Première tokenisation pour obtenir les longueurs
train_inputs = tokenizer(list(train_sentences), padding=True, truncation=True, return_tensors="pt")
test_inputs = tokenizer(list(test_sentences), padding=True, truncation=True, return_tensors="pt")
train_labels_encoded = tokenizer(list(train_labels_text), padding=True, truncation=True, return_tensors="pt")
test_labels_encoded = tokenizer(list(test_labels_text), padding=True, truncation=True, return_tensors="pt")

# Fixer une longueur maximale pour uniformiser les séquences
max_length = max(
    train_inputs['input_ids'].size(1), 
    test_inputs['input_ids'].size(1),
    train_labels_encoded['input_ids'].size(1), 
    test_labels_encoded['input_ids'].size(1)
)

In [44]:

# Retokeniser avec la longueur maximale
train_inputs = tokenizer(list(train_sentences), padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
test_inputs = tokenizer(list(test_sentences), padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
train_labels_encoded = tokenizer(list(train_labels_text), padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
test_labels_encoded = tokenizer(list(test_labels_text), padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")

# Vérifications finales
print("Taille des entrées d'entraînement :", train_inputs['input_ids'].shape)
print("Taille des étiquettes d'entraînement :", train_labels_encoded['input_ids'].shape)
print("Taille des entrées de test :", test_inputs['input_ids'].shape)
print("Taille des étiquettes de test :", test_labels_encoded['input_ids'].shape)

Taille des entrées d'entraînement : torch.Size([70228, 67])
Taille des étiquettes d'entraînement : torch.Size([10207, 67])
Taille des entrées de test : torch.Size([17557, 67])
Taille des étiquettes de test : torch.Size([2536, 67])


In [45]:
# Ajuster les tailles des entrées et des étiquettes pour qu'elles correspondent
train_inputs['input_ids'] = train_inputs['input_ids'][:len(train_labels_encoded['input_ids'])]
test_inputs['input_ids'] = test_inputs['input_ids'][:len(test_labels_encoded['input_ids'])]

# Vérifier à nouveau les tailles après ajustement
print(f"Taille ajustée des entrées d'entraînement : {train_inputs['input_ids'].size(0)}")
print(f"Taille ajustée des étiquettes d'entraînement : {train_labels_encoded['input_ids'].size(0)}")


Taille ajustée des entrées d'entraînement : 10207
Taille ajustée des étiquettes d'entraînement : 10207


In [46]:
# Ajuster les tailles des entrées et des étiquettes du jeu de test pour qu'elles correspondent
test_inputs['input_ids'] = test_inputs['input_ids'][:len(test_labels_encoded['input_ids'])]
test_labels_encoded['input_ids'] = test_labels_encoded['input_ids'][:len(test_inputs['input_ids'])]

# Vérifier les tailles après ajustement
print(f"Taille ajustée des entrées de test : {test_inputs['input_ids'].size(0)}")
print(f"Taille ajustée des étiquettes de test : {test_labels_encoded['input_ids'].size(0)}")


Taille ajustée des entrées de test : 2536
Taille ajustée des étiquettes de test : 2536


In [47]:
from torch.utils.data import TensorDataset, DataLoader

# Créer les jeux de données avec des tailles correspondantes
train_dataset = TensorDataset(train_inputs['input_ids'], train_labels_encoded['input_ids'])
test_dataset = TensorDataset(test_inputs['input_ids'], test_labels_encoded['input_ids'])

# Créer les DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("DataLoader créé avec succès !")

DataLoader créé avec succès !


In [48]:
import torch.optim as optim

# 1. Initialisation du tokenizer (pour la tokenisation)
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# 2. Définition du modèle Seq2SeqLSTM (vous pouvez ajuster selon vos besoins)
class Seq2SeqLSTM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super(Seq2SeqLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.encoder = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.decoder = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, src, tgt):
        # Encoder
        src_embedded = self.embedding(src)
        _, (hidden, cell) = self.encoder(src_embedded)

        # Decoder
        tgt_embedded = self.embedding(tgt)
        outputs, _ = self.decoder(tgt_embedded, (hidden, cell))
        outputs = self.fc(outputs)
        return outputs

# 3. Initialisation du modèle, de la perte et de l'optimiseur
VOCAB_SIZE = tokenizer.vocab_size
EMBED_SIZE = 256
HIDDEN_SIZE = 512

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Créer une instance du modèle et l'envoyer sur le device
model = Seq2SeqLSTM(VOCAB_SIZE, EMBED_SIZE, HIDDEN_SIZE).to(device)

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [49]:
EPOCHS = 3  # Nombre d'époques
model.train()  # Mettre le modèle en mode entraînement
for epoch in range(EPOCHS):
    total_loss = 0
    for batch in train_loader:
        src, tgt = batch
        src, tgt = src.to(device), tgt.to(device)

        optimizer.zero_grad()  # Réinitialiser les gradients
        outputs = model(src, tgt[:, :-1])  # Décalage des cibles pour l'entraînement
        loss = criterion(outputs.reshape(-1, VOCAB_SIZE), tgt[:, 1:].reshape(-1))  # Calcul de la perte

        loss.backward()  # Rétropropagation
        optimizer.step()  # Mise à jour des poids

        total_loss += loss.item()

    print(f"Époque {epoch + 1}/{EPOCHS}, Perte moyenne : {total_loss / len(train_loader)}")


Époque 1/3, Perte moyenne : 5.688161609314826
Époque 2/3, Perte moyenne : 4.519168390375693
Époque 3/3, Perte moyenne : 4.050442573045115


In [50]:
model.eval()  # Mettre le modèle en mode évaluation
total_test_loss = 0

with torch.no_grad():  # Désactive la mise à jour des gradients pendant l'évaluation
    for batch in test_loader:
        src, tgt = batch
        src, tgt = src.to(device), tgt.to(device)

        outputs = model(src, tgt[:, :-1])
        loss = criterion(outputs.reshape(-1, VOCAB_SIZE), tgt[:, 1:].reshape(-1))
        total_test_loss += loss.item()

print(f"Perte moyenne sur le test : {total_test_loss / len(test_loader)}")


Perte moyenne sur le test : 4.380667638778687


In [51]:

VOCAB_SIZE = tokenizer.vocab_size
EMBED_SIZE = 256
HIDDEN_SIZE = 512

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Seq2SeqLSTM(VOCAB_SIZE, EMBED_SIZE, HIDDEN_SIZE).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [52]:

from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(train_inputs['input_ids'], train_labels_encoded['input_ids'])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

EPOCHS = 3

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch in train_loader:
        src, tgt = batch
        src, tgt = src.to(device), tgt.to(device)

        optimizer.zero_grad()
        outputs = model(src, tgt[:, :-1])
        loss = criterion(outputs.reshape(-1, VOCAB_SIZE), tgt[:, 1:].reshape(-1))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss / len(train_loader)}")


Epoch 1/3, Loss: 5.658261033061156
Epoch 2/3, Loss: 4.479279763272563
Epoch 3/3, Loss: 4.0239766144827245


In [55]:

with torch.no_grad():
    for i in range(5):  # Show 5 examples
        src = test_inputs['input_ids'][i].unsqueeze(0).to(device)
        tgt = test_labels_encoded['input_ids'][i].unsqueeze(0).to(device)

        outputs = model(src, tgt[:, :-1])
        predictions = outputs.argmax(dim=-1)

        print("Source Sentence:", tokenizer.decode(src.squeeze(), skip_special_tokens=True))
        print("Expected Translation:", tokenizer.decode(tgt.squeeze(), skip_special_tokens=True))
        print("Predicted Translation:", tokenizer.decode(predictions.squeeze(), skip_special_tokens=True))
        print("-" * 50)


Source Sentence: chnou hia a7ssn blassa saferti liha?
Expected Translation: where is the coolest place you traveled?
Predicted Translation: I is the best??? '????????????
--------------------------------------------------
Source Sentence: sir bniya ou rged m3a l7eya
Expected Translation: Congratulations on your promotion
Predicted Translation: Igratulations thes,
--------------------------------------------------
Source Sentence: jarabt dart chbkat dlbasla obtata
Expected Translation: I don ' t want it
Predicted Translation: I ' ' t have to
--------------------------------------------------
Source Sentence: rah ba9i l7al
Expected Translation: What ingredients?
Predicted Translation: I do you do you???????????????????????????
--------------------------------------------------
Source Sentence: raki sme3ti tbib achnou galik
Expected Translation: it ' s ok
Predicted Translation: I ' s a
--------------------------------------------------


In [57]:
torch.save(model.state_dict(), 'seq2seq_model.pth')
print("Modèle sauvegardé avec succès !")

Modèle sauvegardé avec succès !
